# Data Cleaning
Before we put our data into our model we are going to have to do some tweaks to the data. I am going to do all
the data cleaning and preprocessing in this notebook!

## Packages

In [14]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import numpy as np
from PIL import Image
from io import BytesIO
import tensorflow as tf
import ast

In [15]:
## Reading Data
df = pd.read_csv("Products_Updated.csv")
print(df.head(10))

   Unnamed: 0                                              title  \
0           0                     Forum 84 Low 'Off White Brown'   
1           1                            Forum Low 'White Black'   
2           2                       Forum Low 'White Royal Blue'   
3           3                        Forum Low 'Chalk White Gum'   
4           4                      M&M's x Forum '84 Low 'Brown'   
5           5                             Forum Low 'Dark Witch'   
6           6                      Forum 84 LG 'White Clear Sky'   
7           7                    atmos x Forum Low 'White Black'   
8           8  Forum 84 High 'No Blood No Foul' Jimmy Jazz Ex...   
9           9                        Forum Low 'Cloud White Red'   

                                               image price  
0  https://image.goat.com/transform/v1/attachment...  $107  
1  https://image.goat.com/transform/v1/attachment...   $56  
2  https://image.goat.com/transform/v1/attachment...   $60  
3  http

# Labeling models 
Here is some code to create labels for models, this is going allow us to benchmark, we are also going to add brands too 

In [16]:
df.loc[0:592, 'model'] = 'forum'
df.loc[592:907, 'model'] = 'samba'
df.loc[0:907, 'brand'] = 'Adidas'

df.loc[1414:1450, 'model'] = 'yzySlides'
df.loc[1414:1450, 'brand'] = 'Yeezy'

df.loc[1451:1834, 'model'] = 'AJ3'
df.loc[1834:2125, 'model'] = 'AJ4'
df.loc[1451:2125, 'brand'] = 'AirJordan'

df.loc[2125:2155, 'model'] = 'YeezyRNR'
df.loc[2155:2254, 'model'] = 'Yeezy350'
df.loc[2125:2254, 'brand'] = 'Yeezy'


df.loc[2254:2613, 'model'] = 'AJ5'
df.loc[2613:3159, 'model'] = 'AJ2'
df.loc[3159:3378, 'model'] = 'AJ11'
df.loc[2254:3378, 'brand'] = 'AirJordan'

df.loc[3378:4470, 'model'] = 'Bapesta'
df.loc[3378:4470, 'brand'] = 'Bapesta'

df.loc[4470:4615, 'model'] = 'NB550'
df.loc[4470:4615, 'brand'] = 'NewBalance'

df.loc[4615:5606, 'model'] = 'DunkHigh'
df.loc[4615:5606, 'brand'] = 'Nike'

df.loc[5606:6449, 'model'] = 'StanSmith'
df.loc[5606:6449, 'brand'] = 'Adidas'

df.loc[6449:6646, 'model'] = '2002R'
df.loc[6449:6646, 'brand'] = 'NewBalance'

df.loc[6646:6820, 'model'] = 'AJ12'
df.loc[6820:7109, 'model'] = 'AJ6'
df.loc[7109:8952, 'model'] = 'AJ1'
df.loc[6646:8952, 'brand'] = 'AirJordan'

df.loc[8952:10417, 'model'] = 'AIRMAX90'
df.loc[10417:11505, 'model'] = 'AIRMAX1'
df.loc[8952:11505, 'brand'] = 'Nike'

df.loc[11505:11560, 'model'] = 'YZY700'
df.loc[11505:11560, 'brand'] = 'Yeezy'

df.loc[11560:12960, 'model'] = 'DUNKLOW'
df.loc[11560:12960, 'brand'] = 'Nike'

df.loc[12960:13302, 'model'] = 'NB990'
df.loc[12960:13302, 'brand'] = 'NewBalance'

df.loc[13302:17081, 'model'] = 'af1'
df.loc[13302:17081, 'brand'] = 'Nike'


/var/folders/9q/kg9gzt1s4d36gdnlr8nf75m40000gn/T/ipykernel_12036/1817049365.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'forum' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[0:592, 'model'] = 'forum'
/var/folders/9q/kg9gzt1s4d36gdnlr8nf75m40000gn/T/ipykernel_12036/1817049365.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Adidas' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[0:907, 'brand'] = 'Adidas'


In [17]:
# getting rid of crocs cuz of poor model selection on goat
df.drop(df.index[908:1415], inplace=True)
# removing any jordans that dont contain air
index_ranges = [(1451, 2125), (2254, 3378), (6646, 8952)]
for start, end in index_ranges:
    df.loc[start:end] = df.loc[start:end][df.loc[start:end]['title'].str.contains('Air', na=False)]

In [18]:
# removing some placeholders and fixing data types
df = df.dropna()
df = df[~df['image'].str.contains('data:image/gif;base64,R0lGODlhAQABAIAAAP///wAAACH5BAEAAAAALAAAAAABAAEAAAICRAEAOw')]
df = df.drop(df.columns[0], axis=1)
df['price'] = df['price'].replace('[\$,]', '', regex=True).astype(float)


## Extracting Vectors for our response
Our goal is going to put these prices into a vector but we have to put them into groups before this happens. 
There are two types of the price groups 
1. Unbalanced
 0 - 100
 100 -200
 ...
 1000-inf
2. Balanced
    Quantiles can divide our data up perfectly!
We are going to store each and use our model to see what performs better. 

I am adding additional code for adding the price and brand vectors, Not these are also going to just a unbalanced
class because tensor flow allows us to add weights to our data as well!

In [19]:
balanced_df = df.copy()
model_df = df.copy()
brand_df = df.copy()

Code for one hot encoding both groups

In [20]:
bins = [0, 100, 200, 300, 400, 500, 600, 700, 800, 900, float('inf')]
labels = ['0-100', '100-200', '200-300', '300-400', '400-500', '500-600', '600-700', '700-800', '800-900', '900+']
df['pg'] = pd.cut(df['price'], bins=bins, labels=labels, right=False)
dummies = pd.get_dummies(df['pg'], prefix='price').astype(int)
df = pd.concat([df, dummies], axis=1)

In [21]:
balanced_df['pg_balanced'], quantile_bins = pd.qcut(balanced_df['price'], 10, retbins=True)
balance_dummies = pd.get_dummies(balanced_df['pg_balanced'], prefix='price').astype(int)
quantile_ranges = [f'({quantile_bins[i]}, {quantile_bins[i+1]})' for i in range(len(quantile_bins)-1)]
balanced_df = pd.concat([balanced_df, balance_dummies], axis=1)
balanced_df
quantile_ranges

['(17.0, 93.0)',
 '(93.0, 116.0)',
 '(116.0, 143.0)',
 '(143.0, 177.0)',
 '(177.0, 217.0)',
 '(217.0, 272.0)',
 '(272.0, 340.0)',
 '(340.0, 457.0)',
 '(457.0, 766.0)',
 '(766.0, 100007.0)']

### Class counts
getting counts that can be used with tensorflow to create class weights

In [22]:
dummies = pd.get_dummies(model_df['model'], prefix='Model').astype(int)
model_df = pd.concat([model_df, dummies], axis=1)
dummy_counts = dummies.sum()
print(dummy_counts)

Model_2002R         190
Model_AIRMAX1       822
Model_AIRMAX90     1044
Model_AJ1          1271
Model_AJ11          158
Model_AJ12          116
Model_AJ2           100
Model_AJ3           163
Model_AJ4           138
Model_AJ5           161
Model_AJ6           115
Model_Bapesta       485
Model_DUNKLOW      1113
Model_DunkHigh      822
Model_NB550         137
Model_NB990         280
Model_StanSmith     625
Model_YZY700         54
Model_Yeezy350       93
Model_YeezyRNR       29
Model_af1          2834
Model_forum         491
Model_samba         217
Model_yzySlides      30
dtype: int64


In [23]:
dummies = pd.get_dummies(brand_df['brand'], prefix='Brand').astype(int)
brand_df = pd.concat([brand_df, dummies], axis=1)
brand_df.head(10)
dummy_counts = dummies.sum()
print(dummy_counts)


Brand_Adidas        1333
Brand_AirJordan     2222
Brand_Bapesta        485
Brand_NewBalance     607
Brand_Nike          6635
Brand_Yeezy          206
dtype: int64


### turning these a single vector!

THESE ONE HOT ENCODED VALUES INTO A NUMPY VECTOR

In [24]:
one_hot_columns = df.columns[df.columns.str.startswith('price_')]
df['UNBALANCED_VECTOR'] = df[one_hot_columns].astype(int).to_numpy().tolist()
df

,title,image,price,model,brand,pg,price_0-100,price_100-200,price_200-300,price_300-400,price_400-500,price_500-600,price_600-700,price_700-800,price_800-900,price_900+,UNBALANCED_VECTOR
0,Forum 84 Low 'Off White Brown',https://image.goat.com/transform/v1/attachment...,107.0,forum,Adidas,100-200,0,1,0,0,0,0,0,0,0,0,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0]"
1,Forum Low 'White Black',https://image.goat.com/transform/v1/attachment...,56.0,forum,Adidas,0-100,1,0,0,0,0,0,0,0,0,0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
2,Forum Low 'White Royal Blue',https://image.goat.com/transform/v1/attachment...,60.0,forum,Adidas,0-100,1,0,0,0,0,0,0,0,0,0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
3,Forum Low 'Chalk White Gum',https://image.goat.com/transform/v1/attachment...,73.0,forum,Adidas,0-100,1,0,0,0,0,0,0,0,0,0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
4,M&M's x Forum '84 Low 'Brown',https://image.goat.com/transform/v1/attachment...,114.0,forum,Adidas,100-200,0,1,0,0,0,0,0,0,0,0,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16364,RTFKT x Air Force 1 Low 'Demon',https://image.goat.com/transform/v1/attachment...,932.0,af1,Nike,900+,0,0,0,0,0,0,0,0,0,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1]"
16404,RTFKT x Air Force 1 Low 'Robot',https://image.goat.com/transform/v1/attachment...,1507.0,af1,Nike,900+,0,0,0,0,0,0,0,0,0,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1]"
16405,Air Force 1 Low 'Wear and Tear',https://image.goat.com/transform/v1/attachment...,225.0,af1,Nike,200-300,0,0,1,0,0,0,0,0,0,0,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0]"
16449,Air Force 1 Crater Flyknit 'Wolf Grey' Sample,https://image.goat.com/transform/v1/attachment...,313.0,af1,Nike,300-400,0,0,0,1,0,0,0,0,0,0,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0]"


In [25]:
one_hot_columns = balanced_df.columns[balanced_df.columns.str.startswith('price_')]
balanced_df['BALANCED_VECTOR'] = balanced_df[one_hot_columns].astype(int).to_numpy().tolist()
balanced_df

,title,image,price,model,brand,pg_balanced,"price_(16.999, 93.0]","price_(93.0, 116.0]","price_(116.0, 143.0]","price_(143.0, 177.0]","price_(177.0, 217.0]","price_(217.0, 272.0]","price_(272.0, 340.0]","price_(340.0, 457.0]","price_(457.0, 766.0]","price_(766.0, 100007.0]",BALANCED_VECTOR
0,Forum 84 Low 'Off White Brown',https://image.goat.com/transform/v1/attachment...,107.0,forum,Adidas,"(93.0, 116.0]",0,1,0,0,0,0,0,0,0,0,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0]"
1,Forum Low 'White Black',https://image.goat.com/transform/v1/attachment...,56.0,forum,Adidas,"(16.999, 93.0]",1,0,0,0,0,0,0,0,0,0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
2,Forum Low 'White Royal Blue',https://image.goat.com/transform/v1/attachment...,60.0,forum,Adidas,"(16.999, 93.0]",1,0,0,0,0,0,0,0,0,0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
3,Forum Low 'Chalk White Gum',https://image.goat.com/transform/v1/attachment...,73.0,forum,Adidas,"(16.999, 93.0]",1,0,0,0,0,0,0,0,0,0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
4,M&M's x Forum '84 Low 'Brown',https://image.goat.com/transform/v1/attachment...,114.0,forum,Adidas,"(93.0, 116.0]",0,1,0,0,0,0,0,0,0,0,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16364,RTFKT x Air Force 1 Low 'Demon',https://image.goat.com/transform/v1/attachment...,932.0,af1,Nike,"(766.0, 100007.0]",0,0,0,0,0,0,0,0,0,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1]"
16404,RTFKT x Air Force 1 Low 'Robot',https://image.goat.com/transform/v1/attachment...,1507.0,af1,Nike,"(766.0, 100007.0]",0,0,0,0,0,0,0,0,0,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1]"
16405,Air Force 1 Low 'Wear and Tear',https://image.goat.com/transform/v1/attachment...,225.0,af1,Nike,"(217.0, 272.0]",0,0,0,0,0,1,0,0,0,0,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0]"
16449,Air Force 1 Crater Flyknit 'Wolf Grey' Sample,https://image.goat.com/transform/v1/attachment...,313.0,af1,Nike,"(272.0, 340.0]",0,0,0,0,0,0,1,0,0,0,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0]"


In [26]:
one_hot_columns = brand_df.columns[brand_df.columns.str.startswith('Brand_')]
brand_df['UNBALANCED_BRAND_VECTOR'] = brand_df[one_hot_columns].astype(int).to_numpy().tolist()
brand_df

,title,image,price,model,brand,Brand_Adidas,Brand_AirJordan,Brand_Bapesta,Brand_NewBalance,Brand_Nike,Brand_Yeezy,UNBALANCED_BRAND_VECTOR
0,Forum 84 Low 'Off White Brown',https://image.goat.com/transform/v1/attachment...,107.0,forum,Adidas,1,0,0,0,0,0,"[1, 0, 0, 0, 0, 0]"
1,Forum Low 'White Black',https://image.goat.com/transform/v1/attachment...,56.0,forum,Adidas,1,0,0,0,0,0,"[1, 0, 0, 0, 0, 0]"
2,Forum Low 'White Royal Blue',https://image.goat.com/transform/v1/attachment...,60.0,forum,Adidas,1,0,0,0,0,0,"[1, 0, 0, 0, 0, 0]"
3,Forum Low 'Chalk White Gum',https://image.goat.com/transform/v1/attachment...,73.0,forum,Adidas,1,0,0,0,0,0,"[1, 0, 0, 0, 0, 0]"
4,M&M's x Forum '84 Low 'Brown',https://image.goat.com/transform/v1/attachment...,114.0,forum,Adidas,1,0,0,0,0,0,"[1, 0, 0, 0, 0, 0]"
...,...,...,...,...,...,...,...,...,...,...,...,...
16364,RTFKT x Air Force 1 Low 'Demon',https://image.goat.com/transform/v1/attachment...,932.0,af1,Nike,0,0,0,0,1,0,"[0, 0, 0, 0, 1, 0]"
16404,RTFKT x Air Force 1 Low 'Robot',https://image.goat.com/transform/v1/attachment...,1507.0,af1,Nike,0,0,0,0,1,0,"[0, 0, 0, 0, 1, 0]"
16405,Air Force 1 Low 'Wear and Tear',https://image.goat.com/transform/v1/attachment...,225.0,af1,Nike,0,0,0,0,1,0,"[0, 0, 0, 0, 1, 0]"
16449,Air Force 1 Crater Flyknit 'Wolf Grey' Sample,https://image.goat.com/transform/v1/attachment...,313.0,af1,Nike,0,0,0,0,1,0,"[0, 0, 0, 0, 1, 0]"


In [27]:
one_hot_columns = model_df.columns[model_df.columns.str.startswith('Model_')]
model_df['UNBALANCED_MODEL_VECTOR'] = model_df[one_hot_columns].astype(int).to_numpy().tolist()
model_df
model_df.columns

Index(['title', 'image', 'price', 'model', 'brand', 'Model_2002R',
       'Model_AIRMAX1', 'Model_AIRMAX90', 'Model_AJ1', 'Model_AJ11',
       'Model_AJ12', 'Model_AJ2', 'Model_AJ3', 'Model_AJ4', 'Model_AJ5',
       'Model_AJ6', 'Model_Bapesta', 'Model_DUNKLOW', 'Model_DunkHigh',
       'Model_NB550', 'Model_NB990', 'Model_StanSmith', 'Model_YZY700',
       'Model_Yeezy350', 'Model_YeezyRNR', 'Model_af1', 'Model_forum',
       'Model_samba', 'Model_yzySlides', 'UNBALANCED_MODEL_VECTOR'],
      dtype='object')

Create A subdf holding our contents

In [28]:
df = df.drop_duplicates(subset='image')
final_df = df[['title', 'image', 'price', 'UNBALANCED_VECTOR']]
final_df = final_df.merge(balanced_df[['image', 'BALANCED_VECTOR']], on='image', how='left')
final_df = final_df.merge(model_df[['image', 'UNBALANCED_MODEL_VECTOR']], on='image', how='left')
final_df = final_df.merge(brand_df[['image', 'UNBALANCED_BRAND_VECTOR']], on='image', how='left')
final_df

,title,image,price,UNBALANCED_VECTOR,BALANCED_VECTOR,UNBALANCED_MODEL_VECTOR,UNBALANCED_BRAND_VECTOR
0,Forum 84 Low 'Off White Brown',https://image.goat.com/transform/v1/attachment...,107.0,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 0, 0, 0, 0, 0]"
1,Forum Low 'White Black',https://image.goat.com/transform/v1/attachment...,56.0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 0, 0, 0, 0, 0]"
2,Forum Low 'White Royal Blue',https://image.goat.com/transform/v1/attachment...,60.0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 0, 0, 0, 0, 0]"
3,Forum Low 'Chalk White Gum',https://image.goat.com/transform/v1/attachment...,73.0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 0, 0, 0, 0, 0]"
4,M&M's x Forum '84 Low 'Brown',https://image.goat.com/transform/v1/attachment...,114.0,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 0, 0, 0, 0, 0]"
...,...,...,...,...,...,...,...
11921,RTFKT x Air Force 1 Low 'Demon',https://image.goat.com/transform/v1/attachment...,932.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 1, 0]"
11922,RTFKT x Air Force 1 Low 'Robot',https://image.goat.com/transform/v1/attachment...,1507.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 1, 0]"
11923,Air Force 1 Low 'Wear and Tear',https://image.goat.com/transform/v1/attachment...,225.0,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 1, 0]"
11924,Air Force 1 Crater Flyknit 'Wolf Grey' Sample,https://image.goat.com/transform/v1/attachment...,313.0,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 1, 0]"


# IMAGE PREPROCESSING
    * So reminder this image is a link, and we need to turn it into a tensor of shape (224,224,3)
    * This code is going slow do the nature of downloading images, but that's okay.
    * We can apply other techniques later!
    

## Extracting to binary
i am going to put everything in a tensflow binary file, csvs do not store multi dimensional 
data well, and its going to cause us some problems in the future! These 3 dimenional numpy arrays will be stored as a string instead of a tensor with the csv
while tensorflow binary can store these images very well!
Documentation: https://www.tensorflow.org/tutorials/load_data/tfrecord

In [ ]:
def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))
def _int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=value))

def _float_feature(value):
    """Returns a float_list from a float / double."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=value))
def _string_feature(value):
    """Returns a bytes_list from a string / byte."""
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value.encode()]))
    
def serialize_example(image, title, price, unbalanced_vector, balanced_vector, unbalanced_model_vector, unbalanced_brand_vector):
    feature = {
        'image': _bytes_feature(tf.io.encode_jpeg(image).numpy()),
        'title': _string_feature(title),
        'price': _float_feature([price]),
        'unbalanced_vector': _int64_feature(unbalanced_vector),
        'balanced_vector': _int64_feature(balanced_vector),
        'unbalanced_model_vector': _int64_feature(unbalanced_model_vector),
        'unbalanced_brand_vector': _int64_feature(unbalanced_brand_vector)
    }
    example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
    return example_proto.SerializeToString()

In [ ]:
final_df.tail()

,title,image,price,UNBALANCED_VECTOR,BALANCED_VECTOR,UNBALANCED_MODEL_VECTOR,UNBALANCED_BRAND_VECTOR
11921,RTFKT x Air Force 1 Low 'Demon',https://image.goat.com/transform/v1/attachment...,932.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 1, 0]"
11922,RTFKT x Air Force 1 Low 'Robot',https://image.goat.com/transform/v1/attachment...,1507.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 1, 0]"
11923,Air Force 1 Low 'Wear and Tear',https://image.goat.com/transform/v1/attachment...,225.0,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 1, 0]"
11924,Air Force 1 Crater Flyknit 'Wolf Grey' Sample,https://image.goat.com/transform/v1/attachment...,313.0,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 1, 0]"
11925,DJ Clark Kent x Air Force 1 Low 'The List' Fri...,https://image.goat.com/transform/v1/attachment...,14307.0,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 1, 0]"


In [ ]:
i = 0 
with tf.io.TFRecordWriter('Downloaded_Images_Binary_Brands&ModelsA.tfrecords') as writer:
    for _, row in final_df.iterrows():
        image_url = row['image']
        response = requests.get(image_url)
        image = Image.open(BytesIO(response.content))
        image = image.resize((224, 224))
        image = image.convert("RGB")
        resized_image = tf.cast(image, tf.uint8)
        title = str(row['title'])
        if title == "nan":
            continue
        price = row['price']
        unbalanced_vector = row['UNBALANCED_VECTOR']
        balanced_vector = row['BALANCED_VECTOR']
        unbalanced_model_vector = row['UNBALANCED_MODEL_VECTOR']
        unbalanced_brand_vector = row['UNBALANCED_BRAND_VECTOR']
        example = serialize_example(image, title, price, unbalanced_vector, balanced_vector, unbalanced_model_vector, unbalanced_brand_vector)
        writer.write(example)
        i += 1
        if i % 100 == 0:
            print(i)

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
